# Data Analisys Valorant

## Initial Configs

### Imports

In [3]:
import os
from pyspark.sql import SparkSession
from pyspark.sql.functions import avg, col, expr, count, sum, max, udf, dayofweek, date_format, when, mean, median
from pyspark.sql.types import StringType
import sys
sys.path.append('../src/')
from aws.aws import Aws
import io
import pandas as pd 
import boto3
from datetime import datetime

Python-dotenv could not parse statement starting at line 1


### Creating a Spark Session

In [2]:
spark = SparkSession.builder.appName("ValorantDataAnalysis").getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/05/28 13:51:46 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
spark.conf.set("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")
spark.conf.set("spark.hadoop.fs.s3a.access.key", os.getenv('AWS_ACCESS_KEY_ID'))
spark.conf.set("spark.hadoop.fs.s3a.secret.key", os.getenv('AWS_SECRET_ACCESS_KEY'))

### Instantiating used classes

In [13]:
aws = Aws()

## Load Dataframes

### Utils

In [6]:
def get_files(bucket_name : str, folder_path : str) -> list:
    """"""
    objects = aws.list_objetcs_s3(bucket_name, folder_path)


    return objects

In [7]:
def concat_files_s3(objects):
    """"""

    json_files = [obj['Key'] for obj in objects]

    for file in json_files:

        response = aws.read_s3_v2(bucket_name='s3-tcc-fia-valorant', folder_path=file)
        json_data = response['Body'].read().decode('utf-8')

    return io.StringIO(json_data)

In [8]:
def read_spark(data_io):
    """"""
    data_io = pd.read_csv(data_io)
    return spark.createDataFrame(data_io)

In [9]:
def create_dataframe(bucket_name : str, folder_path : str):
    """"""
    objects = get_files(bucket_name, folder_path)
    data_io = concat_files_s3(objects)
    df = read_spark(data_io)

    return df

In [10]:
def save_dataframe_csv(bucket_name, folder_path, file_name, data, file_format):
    # Convert DataFrame to CSV string
    csv_buffer = io.StringIO()
    data.toPandas().to_csv(csv_buffer, index=False)

    # Retrieve CSV data from buffer
    csv_buffer_value = csv_buffer.getvalue()

    date = datetime.now().strftime("%Y%m%d_%H%M%S")
    file_name = file_name + '_' + date + file_format
    file_path = folder_path + file_name

    # Write CSV string to S3
    s3 = boto3.resource('s3')

    try:
        s3.Object(bucket_name, file_path).put(Body=csv_buffer_value)
        print(f"Data was written to S3://{bucket_name}/{folder_path}")

    except Exception as e:

        print(f"Error: {e}")
    
        return False

### df_matches_summary_data

In [10]:
df_matches_summary_data = create_dataframe('s3-tcc-fia-valorant', 'valorant/raw/summary/matches/')

### df_match_metadata_data

In [11]:
df_match_metadata_data = create_dataframe('s3-tcc-fia-valorant', 'valorant/cleaned/details/metadata/')

### df_round_summary_data

In [12]:
df_round_summary_data = create_dataframe('s3-tcc-fia-valorant', 'valorant/cleaned/details/round_summary/')

### df_player_summary_data

In [13]:
df_player_summary_data = create_dataframe('s3-tcc-fia-valorant', 'valorant/cleaned/details/player_summary/')

### df_player_loadout_data

In [14]:
df_player_loadout_data = create_dataframe('s3-tcc-fia-valorant', 'valorant/cleaned/details/player_loadout/')

### df_player_round_damage_data

In [15]:
df_player_round_damage_data = create_dataframe('s3-tcc-fia-valorant', 'valorant/cleaned/details/player_round_damage/')

/tmp/ipykernel_1380/1347267086.py:3: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  data_io = pd.read_csv(data_io)


### df_player_round_kills_data

In [16]:
df_player_round_kills_data = create_dataframe('s3-tcc-fia-valorant', 'valorant/cleaned/details/player_round_kills/')

/tmp/ipykernel_1380/1347267086.py:3: DtypeWarning: Columns (32) have mixed types. Specify dtype option on import or set low_memory=False.
  data_io = pd.read_csv(data_io)


### df_player_round_data

In [17]:
df_player_round_data = create_dataframe('s3-tcc-fia-valorant', 'valorant/cleaned/details/player_round/')

### df_team_summary_data

In [18]:
df_team_summary_data = create_dataframe('s3-tcc-fia-valorant', 'valorant/cleaned/details/team_summary/')

## Books

### Rename Columns

#### df_team_summary_data

In [19]:
df_team_summary_data = df_team_summary_data.withColumnRenamed('RoundswonValue', 'roundsWonValue')
df_team_summary_data = df_team_summary_data.withColumnRenamed('RoundslostValue', 'roundslostValue')
df_team_summary_data = df_team_summary_data.withColumnRenamed('ScoreValue', 'scoreValue')
df_team_summary_data = df_team_summary_data.withColumnRenamed('KillsValue', 'killsValue')
df_team_summary_data = df_team_summary_data.withColumnRenamed('DeathsValue', 'deathsValue')
df_team_summary_data = df_team_summary_data.withColumnRenamed('AssistsValue', 'assistsValue')
df_team_summary_data = df_team_summary_data.withColumnRenamed('DamageValue', 'damageValue')

#### df_player_round_data

In [20]:
df_player_round_data = df_player_round_data.withColumnRenamed('MatchId', 'matchId')
df_player_round_data = df_player_round_data.withColumnRenamed('Round', 'round')
df_player_round_data = df_player_round_data.withColumnRenamed('StatValue_Assists', 'assistsValue')
df_player_round_data = df_player_round_data.withColumnRenamed('StatValue_Damage', 'damageValue')
df_player_round_data = df_player_round_data.withColumnRenamed('StatValue_Deaths', 'deathsValue')
df_player_round_data = df_player_round_data.withColumnRenamed('StatValue_Deaths', 'deathsValue')
df_player_round_data = df_player_round_data.withColumnRenamed('StatValue_Kdratio', 'kdRatio')
df_player_round_data = df_player_round_data.withColumnRenamed('StatValue_Kills', 'killsValue')
df_player_round_data = df_player_round_data.withColumnRenamed('StatValue_Loadoutvalue', 'loadoutValue')
df_player_round_data = df_player_round_data.withColumnRenamed('StatValue_Remainingcredits', 'remainingCredits')
df_player_round_data = df_player_round_data.withColumnRenamed('StatValue_Score', 'scoreValue')
df_player_round_data = df_player_round_data.withColumnRenamed('StatValue_Spentcredits', 'spentCredits')

#### df_player_loadout_data

In [21]:
df_player_loadout_data = df_player_loadout_data.withColumnRenamed('MatchId', 'matchId')
df_player_loadout_data = df_player_loadout_data.withColumnRenamed('latformUserIdentifier', 'platformUserIdentifier')
df_player_loadout_data = df_player_loadout_data.withColumnRenamed('Loadout', 'loadout')
df_player_loadout_data = df_player_loadout_data.withColumnRenamed('StatName', 'statName')
df_player_loadout_data = df_player_loadout_data.withColumnRenamed('StatValue', 'statValue')

#### df_player_round_kills_data

In [22]:
df_player_round_kills_data = df_player_round_kills_data.withColumnRenamed('MatchId', 'matchId')
df_player_round_kills_data = df_player_round_kills_data.withColumnRenamed('Round', 'round')
df_player_round_kills_data = df_player_round_kills_data.withColumnRenamed('DamageValue', 'damageValue')
df_player_round_kills_data = df_player_round_kills_data.withColumnRenamed('Platforminfo_Platformslug', 'platformInfoSlug')
df_player_round_kills_data = df_player_round_kills_data.withColumnRenamed('Platforminfo_Platformuserid', 'platformInfoUserId')
df_player_round_kills_data = df_player_round_kills_data.withColumnRenamed('Platforminfo_Platformuserhandle', 'platformInfoUserHandle')
df_player_round_kills_data = df_player_round_kills_data.withColumnRenamed('Platforminfo_Platformuseridentifier', 'platformInfoUserIdentifier')
df_player_round_kills_data = df_player_round_kills_data.withColumnRenamed('Platforminfo_Avatarurl', 'avatarUrl')
df_player_round_kills_data = df_player_round_kills_data.withColumnRenamed('Platforminfo_Additionalparameters', 'additionalParameters')
df_player_round_kills_data = df_player_round_kills_data.withColumnRenamed('Opponentplatforminfo_Platformslug', 'opponentPlatformInfoSlug')
df_player_round_kills_data = df_player_round_kills_data.withColumnRenamed('Opponentplatforminfo_Platformuserid', 'opponentPlatformInfoUserId')
df_player_round_kills_data = df_player_round_kills_data.withColumnRenamed('Opponentplatforminfo_Platformuserhandle', 'opponentPlatformInfoUserHandle')
df_player_round_kills_data = df_player_round_kills_data.withColumnRenamed('Opponentplatforminfo_Platformuseridentifier', 'opponentPlatformInfoUserIdentifier')
df_player_round_kills_data = df_player_round_kills_data.withColumnRenamed('Opponentplatforminfo_Avatarurl', 'opponentAvatarUrl')
df_player_round_kills_data = df_player_round_kills_data.withColumnRenamed('Opponentplatforminfo_Additionalparameters', 'opponentAdditionalParameters')
df_player_round_kills_data = df_player_round_kills_data.withColumnRenamed('Opponentlocation_X', 'opponentLocationX')
df_player_round_kills_data = df_player_round_kills_data.withColumnRenamed('Opponentlocation_Y', 'opponentLocationY')
df_player_round_kills_data = df_player_round_kills_data.withColumnRenamed('Playerlocations', 'playerLocations')
df_player_round_kills_data = df_player_round_kills_data.withColumnRenamed('Finishingdamage_Damagetype', 'finishingDamageType')
df_player_round_kills_data = df_player_round_kills_data.withColumnRenamed('Finishingdamage_Damageitem', 'finishingDamageItem')
df_player_round_kills_data = df_player_round_kills_data.withColumnRenamed('Finishingdamage_Issecondaryfiremode', 'finishingDamageIsSecondaryFireMode')
df_player_round_kills_data = df_player_round_kills_data.withColumnRenamed('Weaponimageurl', 'weaponImageUrl')
df_player_round_kills_data = df_player_round_kills_data.withColumnRenamed('Weaponname', 'weaponName')
df_player_round_kills_data = df_player_round_kills_data.withColumnRenamed('Weaponcategory', 'weaponCategory')
df_player_round_kills_data = df_player_round_kills_data.withColumnRenamed('Gametime', 'gameTime')
df_player_round_kills_data = df_player_round_kills_data.withColumnRenamed('Roundtime', 'roundTime')

#### df_player_round_damage_data

In [23]:
df_player_round_damage_data = df_player_round_damage_data.withColumnRenamed('MatchId', 'matchId')
df_player_round_damage_data = df_player_round_damage_data.withColumnRenamed('Round', 'round')
df_player_round_damage_data = df_player_round_damage_data.withColumnRenamed('DamageValue', 'damageValue')
df_player_round_damage_data = df_player_round_damage_data.withColumnRenamed('LegshotsValue', 'legShotsValue')
df_player_round_damage_data = df_player_round_damage_data.withColumnRenamed('BodyshotsValue', 'bodyShotsValue')
df_player_round_damage_data = df_player_round_damage_data.withColumnRenamed('HeadshotsValue', 'headShotsValue')
df_player_round_damage_data = df_player_round_damage_data.withColumnRenamed('HeadshotsValue', 'headShotsValue')

#### df_round_summary_data

In [24]:
df_round_summary_data = df_round_summary_data.withColumnRenamed('Round', 'round')
df_round_summary_data = df_round_summary_data.withColumnRenamed('RoundresultValue', 'roundResultValue')
df_round_summary_data = df_round_summary_data.withColumnRenamed('WinningteamValue', 'winningTeamValue')

#### df_player_summary_data

In [25]:
df_player_summary_data = df_player_summary_data.withColumnRenamed('MatchId', 'matchId')
df_player_summary_data = df_player_summary_data.withColumnRenamed('RankValue', 'rankValue')
df_player_summary_data = df_player_summary_data.withColumnRenamed('CurrrankValue', 'currRankValue')
df_player_summary_data = df_player_summary_data.withColumnRenamed('ScoreValue', 'scoreValue')
df_player_summary_data = df_player_summary_data.withColumnRenamed('ScoreperroundValue', 'scorePerRoundValue')
df_player_summary_data = df_player_summary_data.withColumnRenamed('KillsperroundValue', 'killsPerRoundValue')
df_player_summary_data = df_player_summary_data.withColumnRenamed('KillsValue', 'killsValue')
df_player_summary_data = df_player_summary_data.withColumnRenamed('DeathsValue', 'deathsValue')
df_player_summary_data = df_player_summary_data.withColumnRenamed('AssistsValue', 'assistsValue')
df_player_summary_data = df_player_summary_data.withColumnRenamed('KdratioValue', 'kdRatioValue')
df_player_summary_data = df_player_summary_data.withColumnRenamed('DamageValue', 'damageValue')
df_player_summary_data = df_player_summary_data.withColumnRenamed('DamageperroundValue', 'damagePerRoundValue')
df_player_summary_data = df_player_summary_data.withColumnRenamed('DamagedeltaperroundValue', 'damageDeltaPerRoundValue')
df_player_summary_data = df_player_summary_data.withColumnRenamed('SinglekillsValue', 'singleKillsValue')
df_player_summary_data = df_player_summary_data.withColumnRenamed('DoublekillsValue', 'doubleKillsValue')
df_player_summary_data = df_player_summary_data.withColumnRenamed('TriplekillsValue', 'tripleKillsValue')
df_player_summary_data = df_player_summary_data.withColumnRenamed('QuadrakillsValue', 'quadraKillsValue')
df_player_summary_data = df_player_summary_data.withColumnRenamed('PentakillsValue', 'pentaKillsValue')
df_player_summary_data = df_player_summary_data.withColumnRenamed('MultikillsValue', 'multiKillsValue')
df_player_summary_data = df_player_summary_data.withColumnRenamed('GrenadecastsValue', 'grenadeCastsValue')
df_player_summary_data = df_player_summary_data.withColumnRenamed('Ability1CastsValue', 'ability1CastsValue')
df_player_summary_data = df_player_summary_data.withColumnRenamed('Ability2CastsValue', 'ability2CastsValue')
df_player_summary_data = df_player_summary_data.withColumnRenamed('UltimatecastsValue', 'ultimateCastsValue')
df_player_summary_data = df_player_summary_data.withColumnRenamed('GrenadecastsperroundValue', 'grenadeCastsPerRoundValue')
df_player_summary_data = df_player_summary_data.withColumnRenamed('Ability1CastsperroundValue', 'ability1CastsPerRoundValue')
df_player_summary_data = df_player_summary_data.withColumnRenamed('Ability2CastsperroundValue', 'ability2CastsPerRoundValue')
df_player_summary_data = df_player_summary_data.withColumnRenamed('UltimatecastsperroundValue', 'ultimateCastsPerRoundValue')
df_player_summary_data = df_player_summary_data.withColumnRenamed('PlantsValue', 'plantsValue')
df_player_summary_data = df_player_summary_data.withColumnRenamed('DefusesValue', 'defusesValue')
df_player_summary_data = df_player_summary_data.withColumnRenamed('FirstkillsValue', 'firstKillsValue')
df_player_summary_data = df_player_summary_data.withColumnRenamed('FirstdeathsValue', 'firstDeathsValue')
df_player_summary_data = df_player_summary_data.withColumnRenamed('EsrValue', 'esrValue')
df_player_summary_data = df_player_summary_data.withColumnRenamed('FirstkillsperroundValue', 'firstKillsPerRoundValue')
df_player_summary_data = df_player_summary_data.withColumnRenamed('FirstdeathsperroundValue', 'firstDeathsPerRoundValue')
df_player_summary_data = df_player_summary_data.withColumnRenamed('EconratingValue', 'econRatingValue')
df_player_summary_data = df_player_summary_data.withColumnRenamed('HsaccuracyValue', 'hsAccuracyValue')
df_player_summary_data = df_player_summary_data.withColumnRenamed('KastValue', 'KastValue')
df_player_summary_data = df_player_summary_data.withColumnRenamed('ClutchesValue', 'clutchesValue')
df_player_summary_data = df_player_summary_data.withColumnRenamed('RoundswinpctValue', 'roundsWinPctValue')
df_player_summary_data = df_player_summary_data.withColumnRenamed('TrnperformancescoreValue', 'trnPerformanceScoreValue')
df_player_summary_data = df_player_summary_data.withColumnRenamed('Partyid', 'partyId')
df_player_summary_data = df_player_summary_data.withColumnRenamed('Teamid', 'teamId')
df_player_summary_data = df_player_summary_data.withColumnRenamed('Agentkey', 'agentKey')
df_player_summary_data = df_player_summary_data.withColumnRenamed('Agentname', 'agentName')
df_player_summary_data = df_player_summary_data.withColumnRenamed('Agentcolor', 'agentColor')
df_player_summary_data = df_player_summary_data.withColumnRenamed('Agentimageurl', 'agentImageUrl')
df_player_summary_data = df_player_summary_data.withColumnRenamed('Agentportraiturl', 'agentPortraitUrl')
df_player_summary_data = df_player_summary_data.withColumnRenamed('Countrycode', 'countryCode')
df_player_summary_data = df_player_summary_data.withColumnRenamed('Platforminfo_Platformslug', 'platformInfoSlug')
df_player_summary_data = df_player_summary_data.withColumnRenamed('Platforminfo_Platformuserid', 'PlatformInfoUserId')
df_player_summary_data = df_player_summary_data.withColumnRenamed('Platforminfo_Platformuserhandle', 'platformInfoUserHandle')
df_player_summary_data = df_player_summary_data.withColumnRenamed('Platforminfo_Avatarurl', 'avatarUrl')
df_player_summary_data = df_player_summary_data.withColumnRenamed('Platforminfo_Additionalparameters', 'additionalInfoParameters')

### Matches Summary

In [26]:
df_matches_summary_book = (
    df_matches_summary_data
    .select(
        'userId', 'matchId', 'agentName', 'modeKey', 'modeName', 'modeMaxRounds', 'isAvailable',
        'timestamp', 'metadataResult', 'hasWon', 'result', 'mapName', 'seasonName', 'playtimeValue',
        'roundsPlayedValue', 'roundsWonValue', 'roundsLostValue', 'roundsDisconnectedValue',
        'placementValue', 'scoreValue', 'killsValue', 'deathsValue', 'assistsValue', 'damageValue',
        'damageReceivedValue', 'headshotsValue', 'grenadeCastsValue', 'ability1CastsValue',
        'ability2CastsValue', 'ultimateCastsValue', 'dealtHeadshotsValue', 'dealtBodyshotsValue',
        'dealtLegshotsValue', 'econRatingValue', 'suicidesValue', 'revivedValue', 'firstBloodsValue',
        'firstDeathsValue', 'lastDeathsValue', 'survivedValue', 'tradedValue', 'kastedValue', 'kASTValue',
        'flawlessValue', 'thriftyValue', 'acesValue', 'teamAcesValue', 'clutchesValue', 'clutchesLostValue',
        'plantsValue', 'defusesValue', 'kdRatioValue', 'scorePerRoundValue', 'damagePerRoundValue',
        'headshotsPercentageValue',  'damageDeltaValue', 'damageDeltaPerRoundValue', 'rankValue',
        'trnPerformanceScoreValue',
        )
)

In [27]:
df_matches_summary_book.show()

23/05/28 13:52:46 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+---------------+--------------------+---------+-------+-----------+-------------+-----------+--------------------+--------------+------+-------+--------+----------+-------------+-----------------+--------------+---------------+-----------------------+--------------+----------+----------+-----------+------------+-----------+-------------------+--------------+-----------------+------------------+------------------+------------------+-------------------+-------------------+------------------+---------------+-------------+------------+----------------+----------------+---------------+-------------+-----------+-----------+---------+-------------+------------+---------+-------------+-------------+-----------------+-----------+------------+------------------+------------------+-------------------+------------------------+----------------+------------------------+---------+------------------------+
|         userId|             matchId|agentName|modeKey|   modeName|modeMaxRounds|isAvailable| 

In [28]:
save_dataframe_csv('s3-tcc-fia-valorant', 'valorant/refined/matches-summary-book/', 'matches-summary-book', df_matches_summary_book, '.csv')

Data was written to S3://s3-tcc-fia-valorant/valorant/refined/matches-summary-book/


### Match Metadata

In [29]:
df_match_metadata_book = (
    df_match_metadata_data
    .select(
        'matchId', 'expiryDate', 'modeKey',
        'modeName', 'modeMaxRounds', 'duration',
        'dateStarted', 'rounds', 'isRanked',
        'queueId', 'mapName', 
    )
)

In [30]:
save_dataframe_csv('s3-tcc-fia-valorant', 'valorant/refined/match-metadata-book/', 'match-metadata-book', df_match_metadata_book, '.csv')

Data was written to S3://s3-tcc-fia-valorant/valorant/refined/match-metadata-book/


### Round Summary

In [31]:
df_round_summary_book = (
    df_round_summary_data
    .select(
        'matchId', 'round', 'roundResultValue', 'winningTeamValue', 
    )
)

In [32]:
save_dataframe_csv('s3-tcc-fia-valorant', 'valorant/refined/round-summary-book/', 'round-summary-book', df_round_summary_book, '.csv')

Data was written to S3://s3-tcc-fia-valorant/valorant/refined/round-summary-book/


### Player Summary

In [33]:
df_player_summary_book = (
    df_player_summary_data
    .select(
        'matchId', 'platformUserIdentifier', 'rankValue', 'currRankValue',
        'scoreValue', 'scorePerRoundValue', 'killsPerRoundValue', 'killsValue',
        'deathsValue', 'assistsValue', 'kdRatioValue', 'damageValue', 'damagePerRoundValue',
        'damageDeltaPerRoundValue', 'singleKillsValue', 'doubleKillsValue', 'tripleKillsValue',
        'quadraKillsValue', 'pentaKillsValue', 'multiKillsValue', 'grenadeCastsValue', 'ability1CastsValue',
        'ability2CastsValue', 'ultimateCastsValue', 'grenadeCastsPerRoundValue', 'ability1CastsPerRoundValue',
        'ability2CastsPerRoundValue', 'ultimateCastsPerRoundValue', 'plantsValue', 'defusesValue', 'firstKillsValue',
        'firstDeathsValue', 'esrValue', 'firstKillsPerRoundValue', 'firstDeathsPerRoundValue', 'econRatingValue',
        'hsAccuracyValue', 'KastValue', 'clutchesValue', 'roundsWinPctValue', 'trnPerformanceScoreValue', 'partyId',
        'teamId', 'agentName'
    )
)

In [34]:
save_dataframe_csv('s3-tcc-fia-valorant', 'valorant/refined/player-summary-book/', 'player-summary-book', df_player_summary_book, '.csv')

Data was written to S3://s3-tcc-fia-valorant/valorant/refined/player-summary-book/


### Player Loadout

In [35]:
df_player_loadout_book = (
    df_player_loadout_data
    .select(
        'matchId', 'PlatformUserIdentifier', 'loadout',
        'statName', 'statValue'
    )
)

In [36]:
save_dataframe_csv('s3-tcc-fia-valorant', 'valorant/refined/player-loadout-book/', 'player-loadout-book', df_player_loadout_book, '.csv')

23/05/28 13:52:53 WARN TaskSetManager: Stage 5 contains a task of very large size (1176 KiB). The maximum recommended task size is 1000 KiB.


Data was written to S3://s3-tcc-fia-valorant/valorant/refined/player-loadout-book/


### Player Round Damage

In [37]:
df_player_round_damage_book = (
    df_player_round_damage_data
    .select(
        'matchId', 'platformUserIdentifier', 'opponentPlatformUserIdentifier', 'round',
        'damageValue', 'legShotsValue', 'bodyShotsValue', 
    )
)

In [38]:
save_dataframe_csv('s3-tcc-fia-valorant', 'valorant/refined/player-round-damage-book/', 'player-round-damage-book', df_player_round_damage_book, '.csv')

23/05/28 13:52:56 WARN TaskSetManager: Stage 6 contains a task of very large size (6506 KiB). The maximum recommended task size is 1000 KiB.


Data was written to S3://s3-tcc-fia-valorant/valorant/refined/player-round-damage-book/


### Player Round Kills

In [39]:
df_player_round_kills_book = (
    df_player_round_kills_data
    .select(
        'matchId', 'platformUserIdentifier', 'opponentPlatformUserIdentifier', 'round', 
        'damageValue', 'opponentLocationX', 'opponentLocationY', 'finishingDamageType', 'finishingDamageIsSecondaryFireMode',
        'weaponName', 'weaponCategory', 'gameTime', 'roundTime'
    )
)

In [40]:
save_dataframe_csv('s3-tcc-fia-valorant', 'valorant/refined/player-round-kills-book/', 'player-round-kills-book', df_player_round_kills_book, '.csv')

23/05/28 13:53:00 WARN TaskSetManager: Stage 7 contains a task of very large size (10779 KiB). The maximum recommended task size is 1000 KiB.


Data was written to S3://s3-tcc-fia-valorant/valorant/refined/player-round-kills-book/


### Player Round

In [41]:
df_player_round_book = (
    df_player_round_data
    .select(
        'matchId', 'platformUserIdentifier', 'round', 
        'assistsValue', 'damageValue', 'deathsValue', 
        'kdRatio', 'killsValue', 'loadoutValue', 'remainingCredits',
        'scoreValue', 'spentCredits'
    )
)

In [42]:
save_dataframe_csv('s3-tcc-fia-valorant', 'valorant/refined/player-round-book/', 'player-round-book', df_player_round_book, '.csv')

23/05/28 13:53:03 WARN TaskSetManager: Stage 8 contains a task of very large size (2644 KiB). The maximum recommended task size is 1000 KiB.


Data was written to S3://s3-tcc-fia-valorant/valorant/refined/player-round-book/


### Team Summary

In [43]:
df_team_summary_book = (
    df_team_summary_data
    .select(
        'matchId', 'teamId', 'hasWon', 'roundsWonValue', 'roundslostValue',
        'scoreValue', 'killsValue', 'deathsValue', 'assistsValue', 'damageValue'
    )
)

In [44]:
save_dataframe_csv('s3-tcc-fia-valorant', 'valorant/refined/team-summary-book/', 'team-summary-book', df_team_summary_book, '.csv')

Data was written to S3://s3-tcc-fia-valorant/valorant/refined/team-summary-book/


### Players

In [45]:
df_player_book = df_player_summary_book

In [46]:
df_player_book = (
    df_player_book.alias("p")
    .join(df_team_summary_book.alias("t"), 
          (col("p.matchId") == col("t.matchId")) & 
          (col("p.teamId") == col("t.teamId")),
          "inner")
    .select(
        col("p.*"),
        col("t.hasWon"),
        col("t.roundsWonValue"),
        col("t.roundslostValue"),
        col("t.scoreValue").alias("teamScoreValue"),
        col("t.killsValue").alias("teamKillsValue"),
        col("t.deathsValue").alias("teamDeathsValue"),
        col("t.assistsValue").alias("teamAssistsValue"),
        col("t.damageValue").alias("teamDamageValue")
    )
)

In [47]:
df_player_book = (
    df_player_book.alias("p")
    .join(df_player_round_book.alias("t"), 
          (col("p.matchId") == col("t.matchId")) & 
          (col("p.platformUserIdentifier") == col("t.platformUserIdentifier")),
          "inner")
    .groupBy("p.matchId", "p.platformUserIdentifier")
    .agg(
        sum("t.spentCredits").alias("totalSpentCredits"),
        count("t.round").alias("totalRounds")

    )
    .join(df_player_book, ["matchId", "platformUserIdentifier"], "inner")
)

In [48]:
df_player_book = (
    df_player_book.alias("p")
    .join(df_player_round_kills_book.alias("t"), 
          (col("p.matchId") == col("t.matchId")) & 
          (col("p.platformUserIdentifier") == col("t.platformUserIdentifier")),
          "inner")
    .groupBy("p.matchId", "p.platformUserIdentifier", "t.weaponName")
    .agg(
        count("t.weaponName").alias("totalKillsWeaponName")
    )
    .join(df_player_book, ["matchId", "platformUserIdentifier"], "inner")
)

In [49]:
df_player_round_kills_book_aux = df_player_round_kills_book.groupBy("matchId", "platformUserIdentifier", "weaponName") \
    .agg(count("weaponName").alias("totalKillsWeaponName"))

df_player_book_aux = df_player_book.alias("p") \
    .join(df_player_round_kills_book_aux.alias("t"),
          (col("p.matchId") == col("t.matchId")) &
          (col("p.platformUserIdentifier") == col("t.platformUserIdentifier")),
          "inner") \
    .groupBy("p.matchId", "p.platformUserIdentifier") \
    .agg(
        max(col("t.weaponName")).alias("weaponName"),
        max(col("t.totalKillsWeaponName")).alias("maxTotalKillsWeaponName")
    )

In [50]:
df_player_book = df_player_book.drop("weaponName", "totalKillsWeaponName")
df_player_book = df_player_book.dropDuplicates(["matchId", "platformUserIdentifier"])

In [51]:
df_player_book = (
    df_player_book.alias("p")
    .join(df_player_book_aux.alias("t"), 
          (col("p.matchId") == col("t.matchId")) & 
          (col("p.platformUserIdentifier") == col("t.platformUserIdentifier")),
          "inner")
          .select("p.*", "t.weaponName", "maxTotalKillsWeaponName")
    )

In [52]:
df_player_book = (
    df_player_book.alias("p")
    .join(df_matches_summary_book.alias("t"), 
          (col("p.matchId") == col("t.matchId")),
          "inner")
          .select("p.*",
                   "t.isAvailable",
                   "t.timestamp",
                   "t.seasonName",
                   "t.mapName",
                   "t.playtimeValue",
                   "t.result"
                   )
    )

In [53]:
df_player_book = (
    df_player_book.alias("p")
    .join(df_match_metadata_data.alias("t"), 
          (col("p.matchId") == col("t.matchId")),
          "inner")
          .select("p.*",
                   "t.expiryDate",
                   "t.modeKey",
                   "t.modeName",
                   "t.modeMaxRounds",
                   "t.duration",
                   "t.dateStarted",
                   )
    )

In [54]:
total_players_per_matches =(
     df_player_book
        .groupBy('matchId')
        .agg(count(col('platformUserIdentifier'))
             .alias('totalPlayersMatch'))
             )

In [55]:
df_player_book = (
    df_player_book.alias("p")
    .join(total_players_per_matches.alias("t"), 
          (col("p.matchId") == col("t.matchId")),
          "inner")
          .select("p.*",
                  "t.totalPlayersMatch")
)

In [56]:
total_players_per_matches =(
     df_player_book
        .groupBy('matchId', 'teamId')
        .agg(count(col('teamId'))
             .alias('totalPlayersTeam'))
             )

In [57]:
df_player_book = (
    df_player_book.alias("p")
    .join(total_players_per_matches.alias("t"), 
          (col("p.matchId") == col("t.matchId")) & (col("p.teamId") == col("t.teamId")),
          "inner")
    .select("p.*","t.totalPlayersTeam")
)

In [58]:
df_player_book = df_player_book.withColumn("dateMatch", expr("date(timestamp)"))
df_player_book = df_player_book.withColumn("hourMatch", expr("hour(timestamp)"))
df_player_book = df_player_book.withColumn("minutesMatch", expr("minute(timestamp)"))

In [59]:
to_minutes = udf(lambda x: round(x / 60, 2), StringType())
to_hours = udf(lambda x: round(x / 3600, 2), StringType())

In [60]:
df_player_book = df_player_book.withColumn("playtimeMinutesValue", to_minutes(col("playtimeValue")))
df_player_book = df_player_book.withColumn("playtimeHoursValue", to_hours(col("playtimeValue")))

In [61]:
df_player_book = df_player_book.withColumn("weekDayNumber", dayofweek(col("dateMatch")))
df_player_book = df_player_book.withColumn("weekDay", date_format(col("dateMatch"), "EEEE"))

In [62]:
df_player_book = (
    df_player_book
        .withColumn("rankNumberValue",
                    when(col("rankValue") == "Unranked", 1)
                    .when(col("rankValue") == "Iron 1", 2)
                    .when(col("rankValue") == "Iron 2", 3)
                    .when(col("rankValue") == "Iron 3", 4)
                    .when(col("rankValue") == "Bronze 1", 5)
                    .when(col("rankValue") == "Bronze 2", 6)
                    .when(col("rankValue") == "Bronze 3", 7)
                    .when(col("rankValue") == "Silver 1", 8)
                    .when(col("rankValue") == "Silver 2", 9)
                    .when(col("rankValue") == "Silver 3", 10)
                    .when(col("rankValue") == "Gold 1", 11)
                    .when(col("rankValue") == "Gold 2", 12)
                    .when(col("rankValue") == "Gold 3", 13)
                    .when(col("rankValue") == "Platinum 1", 14)
                    .when(col("rankValue") == "Platinum 2", 15)
                    .when(col("rankValue") == "Platinum 3", 16)
                    .when(col("rankValue") == "Diamond 1", 17)
                    .when(col("rankValue") == "Diamond 2", 18)
                    .when(col("rankValue") == "Diamond 3", 19)
                    .when(col("rankValue") == "Immortal 1", 20)
                    .when(col("rankValue") == "Immortal 2", 21)
                    .when(col("rankValue") == "Immortal 3", 22)
                    .when(col("rankValue") == "Radiant 1", 23)
                    .when(col("rankValue") == "Radiant 2", 24)
                    .when(col("rankValue") == "Radiant 3", 25)
                                    .otherwise(0)))

In [63]:
df_player_book = (
    df_player_book
        .withColumn("currRankNumberValue",
                    when(col("currRankValue") == "Unranked", 1)
                    .when(col("currRankValue") == "Iron 1", 2)
                    .when(col("currRankValue") == "Iron 2", 3)
                    .when(col("currRankValue") == "Iron 3", 4)
                    .when(col("currRankValue") == "Bronze 1", 5)
                    .when(col("currRankValue") == "Bronze 2", 6)
                    .when(col("currRankValue") == "Bronze 3", 7)
                    .when(col("currRankValue") == "Silver 1", 8)
                    .when(col("currRankValue") == "Silver 2", 9)
                    .when(col("currRankValue") == "Silver 3", 10)
                    .when(col("currRankValue") == "Gold 1", 11)
                    .when(col("currRankValue") == "Gold 2", 12)
                    .when(col("currRankValue") == "Gold 3", 13)
                    .when(col("currRankValue") == "Platinum 1", 14)
                    .when(col("currRankValue") == "Platinum 2", 15)
                    .when(col("currRankValue") == "Platinum 3", 16)
                    .when(col("currRankValue") == "Diamond 1", 17)
                    .when(col("currRankValue") == "Diamond 2", 18)
                    .when(col("currRankValue") == "Diamond 3", 19)
                    .when(col("currRankValue") == "Immortal 1", 20)
                    .when(col("currRankValue") == "Immortal 2", 21)
                    .when(col("currRankValue") == "Immortal 3", 22)
                    .when(col("currRankValue") == "Radiant 1", 23)
                    .when(col("currRankValue") == "Radiant 2", 24)
                    .when(col("currRankValue") == "Radiant 3", 25)
                                    .otherwise(0)))

In [64]:
df_player_book = (
    df_player_book
        .withColumn("teamNumberId",
                    when(col("teamId") == "Red", 1)
                    .when(col("teamId") == "Blue", 2)
                                    .otherwise(0)))

In [65]:
df_player_book = (
    df_player_book
        .withColumn("teamNumberId",
                    when(col("teamId") == "Red", 1)
                    .when(col("teamId") == "Blue", 2)
                                    .otherwise(0)))

In [66]:
df_player_book = (
    df_player_book
        .withColumn("agentNameNumber",
                    when(col("agentName") == "Astra", 1)
                    .when(col("agentName") == "Phoenix", 2)
                    .when(col("agentName") == "Raze", 3)
                    .when(col("agentName") == "Harbor", 4)
                    .when(col("agentName") == "Gekko", 5)
                    .when(col("agentName") == "Reyna", 6)
                    .when(col("agentName") == "Neon", 7)
                    .when(col("agentName") == "Breach", 8)
                    .when(col("agentName") == "Sova", 9)
                    .when(col("agentName") == "Killjoy", 10)
                    .when(col("agentName") == "Sage", 11)
                    .when(col("agentName") == "Cypher", 12)
                    .when(col("agentName") == "Yoru", 13)
                    .when(col("agentName") == "Skye", 14)
                    .when(col("agentName") == "Brimstone", 15)
                    .when(col("agentName") == "Fade", 16)
                    .when(col("agentName") == "Chamber", 17)
                    .when(col("agentName") == "KAY/O", 18)
                    .when(col("agentName") == "Omen", 19)
                    .when(col("agentName") == "Jett", 20)
                                    .otherwise(0)))

In [67]:
df_player_book = (
    df_player_book
        .withColumn("resultNumber",
                    when(col("result") == "tied", 1)
                    .when(col("result") == "victory", 2)
                    .when(col("result") == "defeat", 3)
                    .otherwise(0)))

In [68]:
df_player_book = (
    df_player_book
        .withColumn("weaponNameNumber",
                    when(col("weaponName") == "Phantom", 1)
                    .when(col("weaponName") == "Vandal", 2)
                    .when(col("weaponName") == "Sheriff", 3)
                    .when(col("weaponName") == "Operator", 4)
                    .when(col("weaponName") == "Ghost", 5)
                    .when(col("weaponName") == "Marshal", 6)
                    .when(col("weaponName") == "Spectre", 7)
                    .when(col("weaponName") == "Shorty", 8)
                    .when(col("weaponName") == "Stinger", 9)
                    .when(col("weaponName") == "Guardian", 10)
                    .when(col("weaponName") == "Judge", 11)
                    .when(col("weaponName") == "Ares", 12)
                    .when(col("weaponName") == "Classic", 13)
                    .when(col("weaponName") == "Odin", 14)
                    .otherwise(0)))

In [69]:
df_player_book = (
    df_player_book
        .withColumn("isAvailableNumber",
                    when(col("isAvailable") == "true", 1)
                    .when(col("isAvailable") == "false", 2)
                    .otherwise(0)))

In [70]:
df_player_book = (
    df_player_book
        .withColumn("mapNameNumber",
                    when(col("mapName") == "Ascent", 1)
                    .when(col("mapName") == "Split", 2)
                    .when(col("mapName") == "Haven", 3)
                    .when(col("mapName") == "Fracture", 4)
                    .when(col("mapName") == "Breeze", 5)
                    .when(col("mapName") == "Pearl", 6)
                    .when(col("mapName") == "Icebox", 7)
                    .when(col("mapName") == "Lotus", 8)
                    .when(col("mapName") == "Bind", 9)
                    .otherwise(0)))

In [71]:
df_player_book = (
    df_player_book
        .withColumn("modeKeyNumber",
                    when(col("modeKey") == "bomb", 1)
                    .otherwise(0)))

In [72]:
df_player_book = (
    df_player_book
        .withColumn("modeNameNumber",
                    when(col("modeName") == "Competitive", 1)
                    .otherwise(0)))

In [73]:
df_player_book = (
    df_player_book
       .withColumn('avgSpentCredits', (col('totalSpentCredits') / col('totalRounds')).cast('int'))
       .withColumn('avgScoreValue', (col('scoreValue') / col('totalRounds')).cast('double'))
       .withColumn('avgKillsValue', (col('killsValue') / col('totalRounds')).cast('double'))
       .withColumn('avgDeathsValue', (col('deathsValue') / col('totalRounds')).cast('double'))
       .withColumn('avgAssistsValue', (col('assistsValue') / col('totalRounds')).cast('double'))
       .withColumn('avgDamageValue', (col('damageValue') / col('totalRounds')).cast('double'))
       .withColumn('avgGrenadeCastsValue', (col('grenadeCastsValue') / col('totalRounds')).cast('double'))
       .withColumn('avgAbility1CastsValue', (col('ability1CastsValue') / col('totalRounds')).cast('double'))
       .withColumn('avgAbility2CastsValue', (col('ability2CastsValue') / col('totalRounds')).cast('double'))
       .withColumn('avgUltimateCastsValue', (col('ultimateCastsValue') / col('totalRounds')).cast('double'))
       .withColumn('avgPlantsValue', (col('plantsValue') / col('totalRounds')).cast('double'))
       .withColumn('avgDefusesValue', (col('defusesValue') / col('totalRounds')).cast('double'))
       .withColumn('avgFirstKillsValue', (col('firstKillsValue') / col('totalRounds')).cast('double'))
       .withColumn('avgFirstDeathsValue', (col('firstDeathsValue') / col('totalRounds')).cast('double'))
       .withColumn('avgEsrValue', (col('esrValue') / col('totalRounds')).cast('double'))
       .withColumn('avgEconRatingValue', (col('econRatingValue') / col('totalRounds')).cast('double'))
       .withColumn('avgKastValue', (col('KastValue') / col('totalRounds')).cast('double'))
       .withColumn('avgClutchesValue', (col('clutchesValue') / col('totalRounds')).cast('double'))
       .withColumn('avgTrnPerformanceScoreValue', (col('trnPerformanceScoreValue') / col('totalRounds')).cast('double'))
       .withColumn('avgTeamScoreValue', (col('teamScoreValue') / col('totalRounds')).cast('double'))
       .withColumn('avgTeamKillsValue', (col('teamKillsValue') / col('totalRounds')).cast('double'))
       .withColumn('avgTeamDeathsValue', (col('teamDeathsValue') / col('totalRounds')).cast('double'))
       .withColumn('avgTeamAssistsValue', (col('teamAssistsValue') / col('totalRounds')).cast('double'))
       .withColumn('avgTeamDamageValue', (col('teamDamageValue') / col('totalRounds')).cast('double'))
)

In [74]:
df_player_book = (
    df_player_book
      .withColumnRenamed('matchId', 'match_id')
      .withColumnRenamed('platformUserIdentifier', 'platform_user_identifier')
      .withColumnRenamed('totalSpentCredits', 'total_spent_credits')
      .withColumnRenamed('totalRounds', 'total_rounds')
      .withColumnRenamed('rankValue', 'rank_value')
      .withColumnRenamed('currRankValue', 'current_rank_value')
      .withColumnRenamed('scoreValue', 'score_value')
      .withColumnRenamed('scorePerRoundValue', 'score_per_round_value')
      .withColumnRenamed('killsPerRoundValue', 'kills_per_round_value')
      .withColumnRenamed('killsValue', 'kills_value')
      .withColumnRenamed('deathsValue', 'deaths_value')
      .withColumnRenamed('assistsValue', 'assists_value')
      .withColumnRenamed('kdRatioValue', 'kd_ratio_value')
      .withColumnRenamed('damageValue', 'damage_value')
      .withColumnRenamed('damagePerRoundValue', 'damage_per_round_value')
      .withColumnRenamed('damageDeltaPerRoundValue', 'damage_delta_per_round_value')
      .withColumnRenamed('singleKillsValue', 'single_kills_value')
      .withColumnRenamed('doubleKillsValue', 'double_kills_value')
      .withColumnRenamed('tripleKillsValue', 'triple_kills_value')
      .withColumnRenamed('quadraKillsValue', 'quadra_kills_value')
      .withColumnRenamed('pentaKillsValue', 'penta_kills_value')
      .withColumnRenamed('multiKillsValue', 'multi_kills_value')
      .withColumnRenamed('grenadeCastsValue', 'grenade_casts_value')
      .withColumnRenamed('ability1CastsValue', 'ability1_casts_value')
      .withColumnRenamed('ability2CastsValue', 'ability2_casts_value')
      .withColumnRenamed('ultimateCastsValue', 'ultimate_casts_value')
      .withColumnRenamed('grenadeCastsPerRoundValue', 'grenade_casts_per_round_value')
      .withColumnRenamed('ability1CastsPerRoundValue', 'ability1_casts_per_round_value')
      .withColumnRenamed('ability2CastsPerRoundValue', 'ability2_casts_per_round_value')
      .withColumnRenamed('ultimateCastsPerRoundValue', 'ultimate_casts_per_round_value')
      .withColumnRenamed('plantsValue', 'plants_value')
      .withColumnRenamed('defusesValue', 'defuses_value')
      .withColumnRenamed('firstKillsValue', 'first_kills_value')
      .withColumnRenamed('firstDeathsValue', 'first_deaths_value')
      .withColumnRenamed('esrValue', 'esr_value')
      .withColumnRenamed('firstKillsPerRoundValue', 'first_kills_per_round_value')
      .withColumnRenamed('firstDeathsPerRoundValue', 'first_deaths_per_round_value')
      .withColumnRenamed('econRatingValue', 'econ_rating_value')
      .withColumnRenamed('hsAccuracyValue', 'hs_accuracy_value')
      .withColumnRenamed('KastValue', 'kast_value')
      .withColumnRenamed('clutchesValue', 'clutches_value')
      .withColumnRenamed('roundsWinPctValue', 'rounds_win_pct_value')
      .withColumnRenamed('trnPerformanceScoreValue', 'trn_performance_score_value')
      .withColumnRenamed('partyId', 'party_id')
      .withColumnRenamed('teamId', 'team_id')
      .withColumnRenamed('agentName', 'agent_name')
      .withColumnRenamed('hasWon', 'has_won')
      .withColumnRenamed('roundsWonValue', 'rounds_won_value')
      .withColumnRenamed('roundslostValue', 'rounds_lost_value')
      .withColumnRenamed('teamScoreValue', 'team_score_value')
      .withColumnRenamed('teamKillsValue', 'team_kills_value')
      .withColumnRenamed('teamDeathsValue', 'team_deaths_value')
      .withColumnRenamed('teamAssistsValue', 'team_assists_value')
      .withColumnRenamed('teamDamageValue', 'team_damage_value')
      .withColumnRenamed('weaponName', 'weapon_name')
      .withColumnRenamed('maxTotalKillsWeaponName', 'max_total_kills_weapon_name')
      .withColumnRenamed('isAvailable', 'is_available')
      .withColumnRenamed('timestamp', 'timestamp')
      .withColumnRenamed('seasonName', 'season_name')
      .withColumnRenamed('mapName', 'map_name')
      .withColumnRenamed('playtimeValue', 'playtime_value')
      .withColumnRenamed('result', 'result')
      .withColumnRenamed('expiryDate', 'expiry_date')
      .withColumnRenamed('modeKey', 'mode_key')
      .withColumnRenamed('modeName', 'mode_name')
      .withColumnRenamed('modeMaxRounds', 'mode_max_rounds')
      .withColumnRenamed('duration', 'duration')
      .withColumnRenamed('dateStarted', 'date_started')
      .withColumnRenamed('totalPlayersMatch', 'total_players_match')
      .withColumnRenamed('totalPlayersTeam', 'total_players_team')
      .withColumnRenamed('dateMatch', 'date_match')
      .withColumnRenamed('hourMatch', 'hour_match')
      .withColumnRenamed('minutesMatch', 'minutes_match')
      .withColumnRenamed('playtimeMinutesValue', 'playtime_minutes_value')
      .withColumnRenamed('playtimeHoursValue', 'playtime_hours_value')
      .withColumnRenamed('weekDayNumber', 'week_day_number')
      .withColumnRenamed('weekDay', 'week_day')
      .withColumnRenamed('rankNumberValue', 'rank_number_value')
      .withColumnRenamed('currRankNumberValue', 'current_rank_number_value')
      .withColumnRenamed('teamNumberId', 'team_number_id')
      .withColumnRenamed('agentNameNumber', 'agent_name_number')
      .withColumnRenamed('resultNumber', 'result_number')
      .withColumnRenamed('weaponNameNumber', 'weapon_name_number')
      .withColumnRenamed('isAvailableNumber', 'is_available_number')
      .withColumnRenamed('mapNameNumber', 'map_name_number')
      .withColumnRenamed('modeKeyNumber', 'mode_key_number')
      .withColumnRenamed('modeNameNumber', 'mode_name_number')
      .withColumnRenamed('avgSpentCredits', 'avg_spent_credits')
      .withColumnRenamed('avgScoreValue', 'avg_score_value')
      .withColumnRenamed('avgKillsValue', 'avg_kills_value')
      .withColumnRenamed('avgDeathsValue', 'avg_deaths_value')
      .withColumnRenamed('avgAssistsValue', 'avg_assists_value')
      .withColumnRenamed('avgDamageValue', 'avg_damage_value')
      .withColumnRenamed('avgGrenadeCastsValue', 'avg_grenade_casts_value')
      .withColumnRenamed('avgAbility1CastsValue', 'avg_ability1_casts_value')
      .withColumnRenamed('avgAbility2CastsValue', 'avg_ability2_casts_value')
      .withColumnRenamed('avgUltimateCastsValue', 'avg_ultimate_casts_value')
      .withColumnRenamed('avgPlantsValue', 'avg_plants_value')
      .withColumnRenamed('avgDefusesValue', 'avg_defuses_value')
      .withColumnRenamed('avgFirstKillsValue', 'avg_first_kills_value')
      .withColumnRenamed('avgFirstDeathsValue', 'avg_first_deaths_value')
      .withColumnRenamed('avgEsrValue', 'avg_esr_value')
      .withColumnRenamed('avgEconRatingValue', 'avg_econ_rating_value')
      .withColumnRenamed('avgKastValue', 'avg_kast_value')
      .withColumnRenamed('avgClutchesValue', 'avg_clutches_value')
      .withColumnRenamed('avgTrnPerformanceScoreValue', 'avg_trn_performance_score_value')
      .withColumnRenamed('avgTeamScoreValue', 'avg_team_score_value')
      .withColumnRenamed('avgTeamKillsValue', 'avg_team_kills_value')
      .withColumnRenamed('avgTeamDeathsValue', 'avg_team_deaths_value')
      .withColumnRenamed('avgTeamAssistsValue', 'avg_team_assists_value')
      .withColumnRenamed('avgTeamDamageValue', 'avg_team_damage_value')
)

In [75]:
save_dataframe_csv('s3-tcc-fia-valorant', 'valorant/refined/player-book/', 'player-book', df_player_book, '.csv')

23/05/28 13:54:21 WARN TaskSetManager: Stage 12 contains a task of very large size (2644 KiB). The maximum recommended task size is 1000 KiB.
23/05/28 13:54:22 WARN TaskSetManager: Stage 13 contains a task of very large size (10779 KiB). The maximum recommended task size is 1000 KiB.
23/05/28 13:54:23 WARN TaskSetManager: Stage 14 contains a task of very large size (2644 KiB). The maximum recommended task size is 1000 KiB.
23/05/28 13:54:24 WARN TaskSetManager: Stage 17 contains a task of very large size (10779 KiB). The maximum recommended task size is 1000 KiB.
23/05/28 13:54:25 WARN TaskSetManager: Stage 20 contains a task of very large size (10779 KiB). The maximum recommended task size is 1000 KiB.


Data was written to S3://s3-tcc-fia-valorant/valorant/refined/player-book/
